## Power transmission line impedance calculator

For three-phase, balanced transmission systems.  Reports positive, negative, and zero sequence symmetrical components of line impedance.  Lines are positioned as shown below with x and y coordinates and resistivity is selected based on type of earth below the line.  Method is based on Power System Analysis and Design [1] pages 197 on.  Earth is replaced with earth return conductors as shown in the Glover textbook referencing Carson [2]

<img src="images/transmission line positions.png" alt="diagram" style="width:300px;"> <img src="images/earth resistivities.png" alt="earth resistivities" style="width:500px">

[1] J. D. Glover, T. J. Overbye, and M. S. Sarma, Power System Analysis & Design. Boston, MA: Cengage Learning, 2017. 

[2] John R. Carson, “Wave Propagation in Overhead Wires with Ground
Return,” Bell System Tech. J. 5 (1926): 539–554.

In [1]:
import numpy as np
import cmath

m_to_ft = lambda m : m * 3.28084
ft_to_m = lambda ft : ft * 0.30479999
mi_to_m = lambda mi : mi * 1.60934 * 1000
km_to_mi = lambda km : km * 0.621371
dist = lambda x1,y1,x2,y2 : np.sqrt((x1-x2)**2 + (y1-y2)**2)

# System parameters
f = 60 # Hz

# Condcutor positions (feet)
apos = 0, 48
bpos = 25, 48
cpos = 50, 48
n1pos = 10, 48+23
n2pos = 40, 48+23

# Conductor properties
Ra = Rb = Rc = 0.1128           # Ohms per mile per conductor
GMRa = 0.0403                   # Geometric mean radius, phase (feet)
ODa = 1.196                     # Conductor outer diameter (inches)
bundle = 2                      # Conductors in bundle
bundle_spacing = 1.5            # feet

# Neutral conductor properties
Rn1 = Rn2 = 1.52                # Ohms per km
GMRn1 = GMRn2 = 0.0021          # feet
ODn = 0.3937                    # Neutral outer diameter (inches)

# values for dry earth
Dkkp = m_to_ft(2690)    # feet

In [4]:
Rkp = 9.869E-7*f
Rk = (Ra / mi_to_m(1)) / bundle     # convert to ohms per meter per phase
Rn = (Rn1 / 1000)                   # convert to ohms per meter

Dkkc = (GMRa * bundle_spacing**(bundle-1))**(1/(bundle))
if bundle >= 4:
    Dkkc = Dkkc * 1.091
Dkkg = GMRn1 

Zaa = Zbb = Zcc = Rk + Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log(Dkkp/Dkkc))
Zn1n1 = Zn2n2 = Rn +   Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log(Dkkp/GMRn1))

Zab =                  Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], bpos[0], bpos[1]-Dkkp))/(dist(apos[0],apos[1],bpos[0],bpos[1]))))
Zbc =                  Rkp + (1j*(1*np.pi*60)*(2E-7)*np.log((dist(bpos[0], bpos[1], cpos[0], cpos[1]-Dkkp))/(dist(bpos[0],bpos[1],cpos[0],cpos[1]))))
Zac =                  Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], cpos[0], cpos[1]-Dkkp))/(dist(apos[0],apos[1],cpos[0],cpos[1]))))

Zan1 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], n1pos[0], n1pos[1]-Dkkp))/(dist(apos[0],apos[1],n1pos[0],n1pos[1]))))
Zcn2 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(cpos[0], cpos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(cpos[0],cpos[1],n2pos[0],n2pos[1]))))
Zan2 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(apos[0], apos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(apos[0],apos[1],n2pos[0],n2pos[1]))))
Zcn1 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(cpos[0], cpos[1], n1pos[0], n1pos[1]-Dkkp))/(dist(cpos[0],cpos[1],n1pos[0],n1pos[1]))))
Zbn1 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(bpos[0], bpos[1], n1pos[0], n1pos[1]-Dkkp))/(dist(bpos[0],bpos[1],n1pos[0],n1pos[1]))))
Zbn2 =                 Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(bpos[0], bpos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(bpos[0],bpos[1],n2pos[0],n2pos[1]))))

Zn1n2 = Rkp + (1j*(2*np.pi*60)*(2E-7)*np.log((dist(n1pos[0], n1pos[1], n2pos[0], n2pos[1]-Dkkp))/(dist(n1pos[0],n1pos[1],n2pos[0],n2pos[1]))))

Za = [[Zaa, Zab, Zac],
      [Zab, Zbb, Zbc],
      [Zac, Zbc, Zcc]]

Zb = [[Zan1, Zan2],
      [Zbn1, Zbn2],
      [Zcn1, Zcn2]]

Zc = [[Zan1, Zbn1, Zcn1],
      [Zan2, Zbn2, Zcn2]]

Zd = [[Zn1n1, Zn1n2],
      [Zn1n2, Zn2n2]]

Zp = Za-np.matmul(np.matmul(Zb,np.linalg.inv(Zd)),Zc)

Zs = (Zp[0,0]+Zp[1,1]+Zp[2,2])/3
Zm = (Zp[0,1]+Zp[0,2]+Zp[1,2])/3

Zpsym = [[Zs,Zm,Zm],
         [Zm,Zs,Zm],
         [Zm,Zm,Zs]]

a = cmath.rect(1,120*(np.pi/180))
Ainv=[[1,1,1],[1,a,a**2],[1,a**2,a]]
A=[[1,1,1],[1,a**2,a],[1,a,a**2]]

Zshat = (1/3)*np.matmul(np.matmul(Ainv,Zpsym),A)  #ohms per meter
print(f"Z0 = {Zshat[0,0]*1000} ohm/km (zero sequence)")
print(f"Z1 = {Zshat[1,1]*1000} ohm/km (positive sequence)")
print(f"Z2 = {Zshat[2,2]*1000} ohm/km (negative sequence)")

Z0 = (0.45598785298888267+1.0747629113547097j) ohm/km (zero sequence)
Z1 = (0.035667699001408516+0.43929770355174746j) ohm/km (positive sequence)
Z2 = (0.035667699001408516+0.4392977035517475j) ohm/km (negative sequence)


## Shunt admittance

In [5]:
ra = ODa/(2*12)         # inches to feet
rn = ODn/(2*12)

Dsc = (ra * bundle_spacing**(bundle-1))**(1/(bundle))
if bundle >= 4:
    Dkkc = Dkkc * 1.091
Dscn = rn

Paa = (1/(2*np.pi*8.854E-12))*np.log((apos[1]*2)/(Dsc))
Pbb = (1/(2*np.pi*8.854E-12))*np.log((bpos[1]*2)/(Dsc))
Pcc = (1/(2*np.pi*8.854E-12))*np.log((cpos[1]*2)/(Dsc))
Pn1n1 = (1/(2*np.pi*8.854E-12))*np.log((n1pos[1]*2)/(Dscn))
Pn2n2 = (1/(2*np.pi*8.854E-12))*np.log((n2pos[1]*2)/(Dscn))

Pab = (1/(2*np.pi*8.854E-12))*np.log(( dist(apos[0], apos[1], bpos[0], (-1*bpos[1])) )/( dist(apos[0], apos[1], bpos[0], bpos[1] ) ))
Pac = (1/(2*np.pi*8.854E-12))*np.log(( dist(apos[0], apos[1], cpos[0], (-1*cpos[1])) )/( dist(apos[0], apos[1], cpos[0], cpos[1] ) ))
Pbc = (1/(2*np.pi*8.854E-12))*np.log(( dist(bpos[0], bpos[1], cpos[0], (-1*cpos[1])) )/( dist(bpos[0], bpos[1], cpos[0], cpos[1] ) ))

Pan1 = (1/(2*np.pi*8.854E-12))*np.log(( dist(apos[0], apos[1], n1pos[0], (-1*n1pos[1])) )/( dist(apos[0], apos[1], n1pos[0], n1pos[1] ) ))
Pan2 = (1/(2*np.pi*8.854E-12))*np.log(( dist(apos[0], apos[1], n2pos[0], (-1*n2pos[1])) )/( dist(apos[0], apos[1], n2pos[0], n2pos[1] ) ))
Pbn1 = (1/(2*np.pi*8.854E-12))*np.log(( dist(bpos[0], bpos[1], n1pos[0], (-1*n1pos[1])) )/( dist(bpos[0], bpos[1], n1pos[0], n1pos[1] ) ))
Pbn2 = (1/(2*np.pi*8.854E-12))*np.log(( dist(bpos[0], bpos[1], n2pos[0], (-1*n2pos[1])) )/( dist(bpos[0], bpos[1], n2pos[0], n2pos[1] ) ))
Pcn1 = (1/(2*np.pi*8.854E-12))*np.log(( dist(cpos[0], cpos[1], n1pos[0], (-1*n1pos[1])) )/( dist(cpos[0], cpos[1], n1pos[0], n1pos[1] ) ))
Pcn2 = (1/(2*np.pi*8.854E-12))*np.log(( dist(cpos[0], cpos[1], n2pos[0], (-1*n2pos[1])) )/( dist(cpos[0], cpos[1], n2pos[0], n2pos[1] ) ))

Pn1n2 = (1/(2*np.pi*8.854E-12))*np.log(( dist(n1pos[0], n1pos[1], n2pos[0], (-1*n2pos[1])) )/( dist(n1pos[0], n1pos[1], n2pos[0], n2pos[1] ) ))

Pa = [[Paa, Pab, Pac],
      [Pab, Pbb, Pbc],
      [Pac, Pbc, Pcc]]

Pb = [[Pan1, Pan2],
      [Pbn1, Pbn2],
      [Pcn1, Pcn2]]

Pc = [[Pan1, Pbn1, Pcn1],
      [Pan2, Pbn2, Pcn2]]

Pd = [[Pn1n1, Pn1n2],
      [Pn1n2, Pn2n2]]

Cp = np.linalg.inv(Pa-np.matmul(np.matmul(Pb,np.linalg.inv(Pd)),Pc))

Caa = (Cp[0,0]+Cp[1,1]+Cp[2,2])/3
Cab = (Cp[0,1]+Cp[0,2]+Cp[1,2])/3

Cpsym = [[Caa, Cab, Cab],
       [Cab, Caa, Cab],
       [Cab, Cab, Caa]]

a = cmath.rect(1,120*(np.pi/180))
Ainv=[[1,1,1],[1,a,a**2],[1,a**2,a]]
A=[[1,1,1],[1,a**2,a],[1,a,a**2]]

#Cshat = (1/3)*np.matmul(np.matmul(Ainv,Cpsym),A)  #ohms per meter
#1j*2*np.pi*Cpsym
Cphat = np.multiply(1j*(2*np.pi*f)*1000,(1/3)*np.matmul(np.matmul(Ainv,Cpsym),A))
print(f"Y0 = {Cphat[0,0].imag} ohm/km (zero sequence)")
print(f"Y1 = {Cphat[1,1].imag} ohm/km (positive sequence)")
print(f"Y2 = {Cphat[2,2].imag} ohm/km (negative sequence)")

Y0 = 2.9199521348679816e-06 ohm/km (zero sequence)
Y1 = 4.533700856570196e-06 ohm/km (positive sequence)
Y2 = 4.533700856570196e-06 ohm/km (negative sequence)
